In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.layers import concatenate, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# seed
import os
seed = 123
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.set_random_seed(seed)

In [2]:
train = np.load('data/train.npy', allow_pickle = 'True')
test = np.load('data/test.npy', allow_pickle = 'True')

In [3]:
x = train[:,2:]
x = np.reshape(x, (-1, 28, 28, 1))

x_letter = train[:,1]
x_letter = np.reshape(x_letter, (-1, 1))
en = OneHotEncoder()
x_letter = en.fit_transform(x_letter).toarray()

y = train[:,0]
y = np.reshape(y, (-1, 1))
en = OneHotEncoder()
y = en.fit_transform(y).toarray()

print(x.shape)
print(x_letter.shape)
print(y.shape)

(2048, 28, 28, 1)
(2048, 26)
(2048, 10)


In [4]:
image_generator = ImageDataGenerator(width_shift_range=0.1,
                                     height_shift_range=0.1, 
                                     zoom_range=[0.8,1.2],
                                     shear_range=10)

In [5]:
x_total = x.copy()
def augment(x):
    aug_list = []
    for i in range(x.shape[0]):
        num_aug = 0
        tmp = x[i]
        tmp = tmp.reshape((1,) + tmp.shape)
        for x_aug in image_generator.flow(tmp, batch_size = 1) :
            if num_aug >= 1:
                break
            aug_list.append(x_aug[0])
            num_aug += 1
    aug_list = np.array(aug_list)
    return aug_list

n = 2
for i in range(n):
    arr = augment(x)
    x_total = np.concatenate((x_total, arr), axis=0)
    if i > n:
        break

print(x_total.shape)

(6144, 28, 28, 1)


In [6]:
y_total = y.copy()
for i in range(n):
    arr = y.copy()
    y_total = np.concatenate((y_total, arr), axis=0)

print(y_total.shape)

(6144, 10)


In [7]:
x_letter_total = x_letter.copy()
for i in range(n):
    arr = x_letter.copy()
    x_letter_total = np.concatenate((x_letter_total, arr), axis=0)
    
print(x_letter_total.shape)

(6144, 26)


In [8]:
x_train, x_val, y_train, y_val = train_test_split(x_total, y_total, test_size=0.2, shuffle=True)#, stratify=y_total)
x_letter_train = x_letter_total[:x_train.shape[0],:]
x_letter_val = x_letter_total[x_train.shape[0]:,:]

print(x_train.shape)
print(x_val.shape)
print(y_train.shape)
print(y_val.shape)
print(x_letter_train.shape)
print(x_letter_val.shape)

(4915, 28, 28, 1)
(1229, 28, 28, 1)
(4915, 10)
(1229, 10)
(4915, 26)
(1229, 26)


In [9]:
input1 = Input(shape=(28,28,1))
x1 = Conv2D(64, (3,3), activation='relu', padding='same')(input1)
x1 = Conv2D(64, (3,3), activation='relu', padding='same')(x1)
x1 = Dropout(0.3)(x1)
x1 = MaxPooling2D((2,2))(x1)
x1 = Conv2D(64, (2,2), activation='relu', padding='same')(x1)
x1 = Conv2D(64, (2,2), activation='relu', padding='same')(x1)
x1 = Dropout(0.3)(x1)
x1 = MaxPooling2D((2,2))(x1)
x1 = Conv2D(128, (2,2), activation='relu', padding='same')(x1)
x1 = Conv2D(64, (2,2), activation='relu', padding='same')(x1)
x1 = Dropout(0.3)(x1)
x1 = GlobalAveragePooling2D()(x1)
outputs = Dense(10, activation='softmax')(x1)
#x1 = MaxPooling2D((2,2))(x1)
#x1 = Flatten()(x1)

#input2 = Input(shape=(26,))
#x2 = Dense(50, activation='relu')(input2)
#x2 = Dropout(0.3)(x2)

#merge = concatenate([x1, input2])

#x2 = Dense(500, activation='relu')(merge)
#x2 = Dropout(0.3)(x2)
#x2 = Dense(100, activation='relu')(x1)
#x2 = Dropout(0.3)(x2)
#x2 = Dense(50, activation='relu')(x2)
#x2 = Dropout(0.3)(x2)
#outputs = Dense(10, activation='softmax')(x2)

model = Model(inputs = input1, outputs = outputs)

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 28, 28, 64)        640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 64)        36928     
_________________________________________________________________
dropout (Dropout)            (None, 28, 28, 64)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 64)        16448     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 64)        16448     
__________

In [10]:
model.compile(optimizer = 'adam', metrics = ['accuracy'], loss = 'categorical_crossentropy')

#es = EarlyStopping(monitor='val_loss', patience=20, mode='min', verbose=1)
cp = ModelCheckpoint('./models/{epoch:02d}-{val_acc:.4f}.h5', monitor='val_loss',
                     save_best_only=True, mode='min')

history = model.fit(x_train, y_train,
                    validation_data=(x_val, y_val), 
                    batch_size=64, epochs=100, verbose=1, callbacks = [cp])

Train on 4915 samples, validate on 1229 samples
Epoch 1/100
4915/4915 [==============================] - 5s 1ms/step - loss: 2.4303 - acc: 0.1127 - val_loss: 2.2954 - val_acc: 0.1147
Epoch 2/100
4915/4915 [==============================] - 2s 389us/step - loss: 2.2761 - acc: 0.1392 - val_loss: 2.2670 - val_acc: 0.1440
Epoch 3/100
4915/4915 [==============================] - 2s 374us/step - loss: 2.2081 - acc: 0.1780 - val_loss: 2.2175 - val_acc: 0.2018
Epoch 4/100
4915/4915 [==============================] - 2s 371us/step - loss: 2.1509 - acc: 0.2155 - val_loss: 2.1258 - val_acc: 0.2669
Epoch 5/100
4915/4915 [==============================] - 2s 379us/step - loss: 1.9556 - acc: 0.3003 - val_loss: 1.9760 - val_acc: 0.3491
Epoch 6/100
4915/4915 [==============================] - 2s 374us/step - loss: 1.7936 - acc: 0.3672 - val_loss: 1.7417 - val_acc: 0.4028
Epoch 7/100
4915/4915 [==============================] - 2s 369us/step - loss: 1.6323 - acc: 0.4238 - val_loss: 1.7406 - val_acc: 0.

4915/4915 [==============================] - 2s 374us/step - loss: 0.1232 - acc: 0.9563 - val_loss: 0.5149 - val_acc: 0.8381
Epoch 61/100
4915/4915 [==============================] - 2s 373us/step - loss: 0.1483 - acc: 0.9487 - val_loss: 0.4554 - val_acc: 0.8568
Epoch 62/100
4915/4915 [==============================] - 2s 376us/step - loss: 0.1413 - acc: 0.9491 - val_loss: 0.4969 - val_acc: 0.8430
Epoch 63/100
4915/4915 [==============================] - 2s 365us/step - loss: 0.1282 - acc: 0.9540 - val_loss: 0.4666 - val_acc: 0.8430
Epoch 64/100
4915/4915 [==============================] - 2s 377us/step - loss: 0.1212 - acc: 0.9573 - val_loss: 0.5567 - val_acc: 0.8251
Epoch 65/100
4915/4915 [==============================] - 2s 374us/step - loss: 0.1449 - acc: 0.9512 - val_loss: 0.5035 - val_acc: 0.8381
Epoch 66/100
4915/4915 [==============================] - 2s 372us/step - loss: 0.1343 - acc: 0.9534 - val_loss: 0.4778 - val_acc: 0.8552
Epoch 67/100
4915/4915 [=======================

In [13]:
from tensorflow.keras.models import load_model
best_model = load_model('models/noskipCNN_drop_global_submit.h5')

In [14]:
x_test = test[:,1:]
x_test = np.reshape(x_test, (-1, 28, 28, 1))
print(x_test.shape)

(20480, 28, 28, 1)


In [15]:
submission = pd.read_csv('data/submission.csv')
submission['digit'] = np.argmax(best_model.predict(x_test), axis=1)
submission.to_csv('data/submission_noskip(0821).csv', index=False)